#### Grouping

Three files(fa, acc, ss3) will be used as input files for grouping.

Maybe .fa file name is still like 'Mydata{0}.fa'.

No matter change file name first or not, but our codes not use the name 'Mydata0.fa'. Rather we use 'before_My_0.fa'.

AND DO NOT USE .faa file which is output file from SCRATCH-1D. YOU MUST USE .fa file for grouping.

Before grouping, running below codes. These will make the files into grouping format.

In [8]:
file_index = '2' # index of the files

In [9]:
# fasta file
# Change the file name first : 'Mydata0.fa' into 'before_My_0.fa'
with open('before_My_{}.fa'.format(file_index), 'r') as f_in:
    first_line = f_in.readline().strip()
    if first_line.startswith('>'):
        # Extract the index from the line (excluding '>')
        idx = int(first_line[1:])
        idx_iter = idx

with open('before_My_{}.fa'.format(file_index), 'r') as f_in, open('after_My_{}.fa'.format(file_index), 'w') as f_out:
    #index = 0
    for line in f_in:
        if line.startswith('>'):
            fr = f'>{idx_iter}\n'
            f_out.write(fr)
            idx_iter +=1
        else:
            f_out.write(line)
            
idx_iter = idx

# ACC file
with open('before_My_{}.acc'.format(file_index), 'r') as f_in, open('after_My_{}.acc'.format(file_index), 'w') as f_out:
    # idx=0
    for line in f_in:
        fr = f'>{idx_iter}\n'
        if line.startswith('e') or line.startswith('-'): 
            text = line   
            f_out.write(fr)
            f_out.write(text)
            idx_iter += 1

idx_iter = idx

# SS3 file
with open('before_My_{}.ss3'.format(file_index), 'r') as f_in, open('after_My_{}.ss3'.format(file_index), 'w') as f_out:
    # idx = 0
    cnt = 0
    for line in f_in:
        if cnt % 4 == 0:
            fr = f'>{idx_iter}\n'
            idx_iter +=1
            f_out.write(fr)
        if cnt % 4 == 2:
            text = line
            f_out.write(text)
        
        cnt +=1
        
idx_iter = idx

#### If there are failed data to make ss3 and acc files, you should check the .log file then you can find the failed fasta index
#### input the failed index and run this line with only '.fa' file, 
#### AGAIN, DO NOT RUN THIS CODE TO ss3 and acc files! ONLY FOR '.fa' FILES.

In [1]:
def remove_fasta_entries(input_file, output_file, entry_indices_to_remove):
    global idx_iter
    # Read the Fasta file
    with open(input_file, 'r') as input_fasta:
        lines = input_fasta.readlines()

    # Create a dictionary to store the Fasta entries
    fasta_entries = {}
    current_entry_index = idx_iter
    current_entry_lines = []

    # Process the lines and store entries
    for line in lines:
        if line.startswith('>'):
            if current_entry_index is not None:
                fasta_entries[current_entry_index] = current_entry_lines
            current_entry_index = int(line[1:].strip())
            current_entry_lines = [line]
            #print("current_entry_lines : ", current_entry_lines)
        else:
            current_entry_lines.append(line)

    if current_entry_index is not None:
        fasta_entries[current_entry_index] = current_entry_lines

    # Remove the specified entries
    removed_entries = {}
    for entry_index_to_remove in entry_indices_to_remove:
        if entry_index_to_remove in fasta_entries:
            removed_entries[entry_index_to_remove] = fasta_entries[entry_index_to_remove]

    # Remove the entries from the original dictionary
    for entry_index in removed_entries:
        del fasta_entries[entry_index]
    
    # Update the indices and write the modified Fasta file
    with open(output_file, 'w') as output_fasta:
        for entry_lines in fasta_entries.values():
            updated_entry = f'>{idx_iter}\n'
            updated_entry += ''.join(entry_lines[1:])
            idx_iter += 1
            output_fasta.write(updated_entry)

In [10]:
# Example usage:
input_file = "after_My_2.fa" # the file which you want to remove failed index
output_file = "after_My_2.fa" # If you want to compare with original file, change the output file name.
entry_indices_to_remove = [738] # Again, You can find these failed enetries index in .log file

# [189]
# [22, 53, 121, 128]
# [5,17,33,47,57,64,65,82,95,99,102,106,122,124,129,140,148,155,169,176]
# ... Whatever

remove_fasta_entries(input_file, output_file, entry_indices_to_remove)

Move extracted three 'after' files to DeepAffinity grouping directory

/YOUR/PATH/TO/DeepAffinity/data/script/split_data_script/pfam/our_format

python group.py -a after_My_0.acc -s after_My_0.ss3 -p after_My_0.fa

Then you will get 'protein_grouped_finalPresentation'

#### Do these step manually.

1. Make index prefix for each output file. For example, If you used 0 index files, make output file name into '0_protein_grouped_finalPresentation'

2. Open n_protein_grouped_finalPresentation file and remove the bottom \n(blank) line.

#### After grouping, Make Final DataFrame

In [16]:
import pandas as pd

file_index = '2'

with open('./{}_protein_grouped_finalPresentation'.format(file_index), 'r') as f:
    lines = f.read().split('\n')

AA = []
SPS = []

for i, line in enumerate(lines):
    if i % 2 == 0:
        AA.append(line)
    else:
        SPS.append(line)

print(len(AA))
print(len(SPS))

df = pd.DataFrame({'sequence': AA, 'sps': SPS})
df.to_csv('./Final_My_{}.csv'.format(file_index), sep=',', index=False)

249
249


#### Make all Final_My_{}.csv files for each divided files and merge them with your own protein table on protein sequence column.

now you are ready to use your own SPS.